In [ ]:
import os

os.environ['PROJECT_BASE'] = ''

In [1]:
import pandas as pd

from data_tools.analysis.models import DataSet
from data_tools.analysis.pipeline import Pipeline
from data_tools.analysis.steps import (
    BusinessGlossaryGenerator,
    ColumnProfiler,
    DataTypeIdentifierL1,
    DataTypeIdentifierL2,
    TableProfiler,
)


/home/juhel-phanju/Documents/intugle/projects/data-tools/data-tools/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/juhel-phanju/Documents/intugle/projects/data-tools/data-tools/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "schema" in "Source" shadows an attribute in parent "BaseResource"
  warnings.warn(


In [2]:
# 1. Prepare dummy data
df = pd.DataFrame({
    "product_id": [1, 2, 3],
    "product_name": ["Laptop", "Mouse", "Keyboard"],
    "price": [1200.00, 25.00, 75.00],
    "last_updated": pd.to_datetime(["2023-01-01", "2023-01-05", "2023-01-10"]),
})
table_name = "products"
domain = "e-commerce"

pipeline = Pipeline([
    TableProfiler(),
    ColumnProfiler(),
    DataTypeIdentifierL1(),
    DataTypeIdentifierL2(),
    BusinessGlossaryGenerator(domain=domain),
])

# 2. Initialize DataSet
dataset = DataSet(df=df, name=table_name)

# 3. Run prerequisite steps (TableProfiler, ColumnProfiler)
dataset = pipeline.run(df, table_name)



/home/juhel-phanju/Documents/intugle/projects/data-tools/data-tools/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/juhel-phanju/Documents/intugle/projects/data-tools/data-tools/.venv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 4/4 [00:05<00:00,  1.43s/it]


In [3]:

dataset.save_yaml()

# Create pandas dataframe for tables of postgres

In [9]:
import pandas as pd

from data_tools.analysis.models import DataSet
from data_tools.analysis.pipeline import Pipeline
from data_tools.analysis.steps import (
    BusinessGlossaryGenerator,
    ColumnProfiler,
    DataTypeIdentifierL1,
    DataTypeIdentifierL2,
    KeyIdentifier,
    TableProfiler,
)
from data_tools.core.settings import settings
from data_tools.mcp.adapter.psql import PsqlPool



In [2]:
psql = PsqlPool(
    user=settings.POSTGRES_USER,
    password=settings.POSTGRES_PASSWORD,
    host=settings.POSTGRES_HOST,
    database=settings.POSTGRES_DB,
    port=settings.POSTGRES_PORT,
    schema=settings.POSTGRES_SCHEMA,
)

In [3]:
tables = [
    "conditions",
    "claims",
    "claims_transactions",
    "medications",
    "observations",
    "encounters",
    "careplans",
    "immunizations",
    "patients",
    "allergies",
    "providers",
    "organizations"
]

domain = "healthcare"

In [4]:
table_dfs: dict[str, pd.DataFrame] = {}

# Create DataFrame for each table
for table in tables:
    query = f"select * from {table} limit 10000"

    data = await psql.execute(query)
    data = [dict(d) for d in data]

    df = pd.DataFrame(data)

    table_dfs[table] = df

In [5]:
# Initialize the pipeline with the domain
pipeline = Pipeline([
    TableProfiler(),
    ColumnProfiler(),
    DataTypeIdentifierL1(),
    DataTypeIdentifierL2(),
    BusinessGlossaryGenerator(domain=domain),
    KeyIdentifier(),
])


datasets: dict[str, DataSet] = {}
# Run Pipeline for each datafram and save results
for table, df in table_dfs.items():
    ds = pipeline.run(df, table)

    datasets[table] = ds


/home/juhel-phanju/Documents/intugle/projects/data-tools/data-tools/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/juhel-phanju/Documents/intugle/projects/data-tools/data-tools/.venv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
 33%|███▎      | 2/6 [00:00<00:00,  4.00it/s][!] Error while llm response parsing: Got invalid JSON object. Error: Expecting ',' delimiter: line 2 column 27 (char 28)
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
100%|██████████| 6/6 [00:08<00:00,  1.41s/it]
/home/juhel-phanju/Documents/intugle/projects/data-tools/data-tools/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home

In [6]:
# save all datasets metadata to YAML files
for ds in datasets.values():
    ds.save_yaml()

# Run Link prediction and save

In [22]:
from data_tools.link_predictor.predictor import LinkPredictionSaver, LinkPredictor

In [14]:
# Initialize the predictor
predictor = LinkPredictor(list(datasets.values()))

# Run the prediction
results = predictor.predict()

Dataset 'conditions' is missing key identification. Running prerequisite analysis...


/home/juhel-phanju/Documents/intugle/projects/data-tools/data-tools/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/juhel-phanju/Documents/intugle/projects/data-tools/data-tools/.venv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 6/6 [00:02<00:00,  2.07it/s]


Dataset 'claims' already processed. Skipping analysis.
Dataset 'claims_transactions' already processed. Skipping analysis.
Dataset 'medications' is missing key identification. Running prerequisite analysis...


/home/juhel-phanju/Documents/intugle/projects/data-tools/data-tools/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/juhel-phanju/Documents/intugle/projects/data-tools/data-tools/.venv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 13/13 [00:06<00:00,  2.16it/s]

Dataset 'observations' is missing key identification. Running prerequisite analysis...


...

No links found for this pair.

--- Comparing 'allergies' <=> 'organizations' ---


[*] allergies <==> organizations Runs:1: 100%|██████████| 1/1 [00:00<00:00,  1.04it/s]


No links found for this pair.

--- Comparing 'providers' <=> 'organizations' ---


[*] providers <==> organizations Runs:1: 100%|██████████| 1/1 [00:00<00:00,  1.09it/s]

Found 1 potential link(s).


In [17]:
results

LinkPredictionResult(links=[PredictedLink(from_dataset='conditions', from_column='encounter', to_dataset='encounters', to_column='id'), PredictedLink(from_dataset='conditions', from_column='encounter', to_dataset='careplans', to_column='encounter'), PredictedLink(from_dataset='conditions', from_column='patient', to_dataset='patients', to_column='id'), PredictedLink(from_dataset='claims', from_column='id', to_dataset='claims_transactions', to_column='claimid'), PredictedLink(from_dataset='claims', from_column='patientid', to_dataset='patients', to_column='id'), PredictedLink(from_dataset='claims', from_column='providerid', to_dataset='providers', to_column='id'), PredictedLink(from_dataset='claims', from_column='providerid', to_dataset='organizations', to_column='id'), PredictedLink(from_dataset='claims_transactions', from_column='patientid', to_dataset='patients', to_column='id'), PredictedLink(from_dataset='claims_transactions', from_column='providerid', to_dataset='providers', to_col

In [23]:
LinkPredictionSaver.save_yaml(results, "__relationships__.yaml")